In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn  # All neural network modules, nn.Linear, nn.Conv2d, BatchNorm, Loss functions
import torch.optim as optim  # For all Optimization algorithms, SGD, Adam, etc.
import torch.nn.functional as F  # All functions that don't have any parameters
from torch.utils.data import (DataLoader,)  # Gives easier dataset managment and creates mini batches
#import torchvision  # torch package for vision related things
#import torchvision.datasets as datasets  # Has standard datasets we can import in a nice way
#import torchvision.transforms as transforms
import pandas as pd
import random
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer
from sklearn.model_selection import RandomizedSearchCV
from tqdm.auto import tqdm
from transformers import DistilBertTokenizer

# Data Loading
Grabbing the combined dataset and including the summary 

In [2]:
# Split the downloaded Youtube01-Psy.csv file into train, validation and test sets
data = pd.read_csv('SpamDataset/actual_train.csv')
# data = pd.read_csv('TSwift_ShakeItOff_Spam.csv')
data = data.sample(frac=1).reset_index(drop=True)  # Shuffle the data
train_data = data[:int(0.85*len(data))]
val_data = data[int(0.85*len(data)):]

# Tokenize the data using distillbert
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Turn pd dataframe into a tokenized PyTorch dataset that DataLoader can use via the SpamDataset class
class CommentSummaryDataset(torch.utils.data.Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]

        comment_tokens = row['CONTENT'] + ' ' + row['SUMMARY']
        label = row['CLASS']
        return comment_tokens, label
    
    def collate_fn(self, batch):
        comments = [row[0] for row in batch]
        labels = [row[1] for row in batch]
        tokens = tokenizer.batch_encode_plus(comments, truncation=True, padding='max_length', max_length=100, return_tensors='pt')
        return tokens,  torch.tensor(labels)
    
class SummaryCommentDataset(torch.utils.data.Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        comment_tokens = row['SUMMARY'] + ' ' + row['CONTENT'] 
        label = row['CLASS']
        return comment_tokens, label
    
    def collate_fn(self, batch):
        comments = [row[0] for row in batch]
        labels = [row[1] for row in batch]
        tokens = tokenizer.batch_encode_plus(comments, truncation=True, padding='max_length', max_length=100, return_tensors='pt')
        return tokens,  torch.tensor(labels)
    
fw_train_dataset = CommentSummaryDataset(train_data)
fw_val_dataset = CommentSummaryDataset(val_data)

bw_train_dataset = SummaryCommentDataset(train_data)
bw_val_dataset = SummaryCommentDataset(val_data)


# Load Data and collate it
batch_size = 64
fw_train_loader = DataLoader(fw_train_dataset, batch_size=batch_size, shuffle=True, collate_fn=fw_train_dataset.collate_fn)
fw_val_loader = DataLoader(fw_val_dataset, batch_size=batch_size, shuffle=True, collate_fn=fw_val_dataset.collate_fn)

bw_train_loader = DataLoader(bw_train_dataset, batch_size=batch_size, shuffle=True, collate_fn=bw_train_dataset.collate_fn)
bw_val_loader = DataLoader(bw_val_dataset, batch_size=batch_size, shuffle=True, collate_fn=bw_val_dataset.collate_fn)

In [3]:
# Print a random sample from the train_loader to confirm correct loading and tokenization
sample = next(iter(fw_train_loader))
print(sample)
# Print decoded tokens for the sample
decoded = tokenizer.decode(sample[0]['input_ids'][0])
print(decoded)

sample = next(iter(bw_train_loader))
print(sample)
# Print decoded tokens for the sample
decoded = tokenizer.decode(sample[0]['input_ids'][0])
print(decoded)

({'input_ids': tensor([[  101,  1996,  2087,  ..., 24768,  2007,   102],
        [  101, 10166,  6796,  ...,  2299,  1005,   102],
        [  101,  2040,  2842,  ...,     0,     0,     0],
        ...,
        [  101,  4638,  2041,  ...,     0,     0,     0],
        [  101,  4658,  1048,  ...,     0,     0,     0],
        [  101,  2190,  1012,  ...,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}, tensor([0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1,
        0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1,
        1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0]))
[CLS] the most watched video on youtube is psy ’ s “ gangnam style ”, with 2. 1 billion views. psy - gangnam style ( 강남스타일 ) m / v psy's " gangnam style " bec

# Model
- Model 1 - include Like count
- Model 2 - include LLM evaluation of the video 

In [4]:
## The network includes an Embedding layer, an Attention layer, an LSTM layer and a Linear layer
class CombinedLSTM(nn.Module):
    def __init__(self, embedding_dict, embedding_size, hidden_size, num_layers, num_classes, dropout):
        super(CombinedLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.embedding = nn.Embedding(embedding_dict, embedding_size)
        self.dropout = nn.Dropout(dropout)
        self.lstm = nn.LSTM(embedding_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.attention = nn.Linear(hidden_size, 1)
        self.fc = nn.Linear(hidden_size, num_classes)
        
    def forward(self, comment_sequence):
        # comment_sequence: (batch_size, max_seq_length)
        embedding = self.dropout(self.embedding(comment_sequence))
        # embedding shape: (batch_size, max_seq_length, embedding_size)
        output, (h_n, c_n) = self.lstm(embedding)
        # output shape: (batch_size, max_seq_length, hidden_size)
        # h_n shape: (num_layers, batch_size, hidden_size)
        attention_weights = F.softmax(self.attention(output), dim=1)
        # attention_weights shape: (batch_size, max_seq_length, 1)
        out = torch.sum(attention_weights * output, dim=1)
        # out shape: (batch_size, hidden_size)
        out = self.fc(out)
        # out shape: (batch_size, num_classes)
        return out

# Utility Functions to run validation

In [7]:
# Validation
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    predictions = []
    targets = []
    model.eval()

    with torch.no_grad():
        for comments, target in loader:
            comments = {key: value.to(device) for key, value in comments.items()}
            target = target.to(device)

            scores = model(comments['input_ids'])
            _, predicted = scores.max(1)
            # Store decoded original comments, model predictions and actual labels 
            num_correct += (predicted == target).sum()
            num_samples += predicted.size(0)

            predictions.extend(predicted.tolist())
            targets.extend(target.tolist())

    # model.train()
    accuracy = num_correct/num_samples
    return accuracy, predictions, targets

def results(model, train_loader, val_loader):
    train_acc, train_preds, train_targets = check_accuracy(train_loader, model)
    val_acc, val_preds, val_targets = check_accuracy(val_loader, model)

    print(f'Train accuracy: {train_acc}')
    print(f'Validation accuracy: {val_acc}')

    train_cm = confusion_matrix(train_targets, train_preds)
    val_cm = confusion_matrix(val_targets, val_preds)

    print(f'Train Confusion Matrix:\n{train_cm}')
    print(f'Validation Confusion Matrix:\n{val_cm}')

    # Classification report
    train_report = classification_report(train_targets, train_preds)
    val_report = classification_report(val_targets, val_preds)

    print(f'Train Classification Report:\n{train_report}')
    print(f'Validation Classification Report:\n{val_report}')

# Training

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
criterion = nn.CrossEntropyLoss()
# Set the hyperparameters for all the models     # TODO: analyze these by model
embedding_dict = tokenizer.vocab_size
embedding_size = 128
hidden_size = 256
num_layers = 4
num_classes = 2
dropout = 0.4
learning_rate = 0.0007
num_epochs = 40

# Train the forwards model
# Initialize the model, loss function and optimizer
modelFW = CombinedLSTM(embedding_dict, embedding_size, hidden_size, num_layers, num_classes, dropout).to(device)
optimizer = optim.Adam(modelFW.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    modelFW.train()
    for batch in tqdm(fw_train_loader):
        tokens, labels = batch
        comment_sequence = tokens['input_ids'].to(device)
        labels = labels.to(device)
        outputs = modelFW(comment_sequence)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')

# Print the results of the forwards model
results(modelFW, fw_train_loader, fw_val_loader)

# Train the backwards model
# Initialize the model, loss function and optimizer
modelBW = CombinedLSTM(embedding_dict, embedding_size, hidden_size, num_layers, num_classes, dropout).to(device)
optimizer = optim.Adam(modelBW.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    modelBW.train()
    for batch in tqdm(bw_train_loader):
        tokens, labels = batch
        comment_sequence = tokens['input_ids'].to(device)
        labels = labels.to(device)
        outputs = modelBW(comment_sequence)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')

# Print the results of the backwards model
results(modelBW, bw_train_loader, bw_val_loader)




  0%|          | 0/29 [00:00<?, ?it/s]

Epoch 1/15, Loss: 0.5731774568557739


  0%|          | 0/29 [00:00<?, ?it/s]

Epoch 2/15, Loss: 0.48866188526153564


  0%|          | 0/29 [00:00<?, ?it/s]

...

Epoch 13/15, Loss: 0.09651412814855576


  0%|          | 0/29 [00:00<?, ?it/s]

Epoch 14/15, Loss: 0.05209822580218315


  0%|          | 0/29 [00:00<?, ?it/s]

Epoch 15/15, Loss: 0.10749689489603043
Train accuracy: 0.9864351749420166
Validation accuracy: 0.9263803958892822
Train Confusion Matrix:
[[989   6]
 [ 19 829]]
Validation Confusion Matrix:
[[172  13]
 [ 11 130]]
Train Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.99      0.99       995
           1       0.99      0.98      0.99       848

    accuracy                           0.99      1843
   macro avg       0.99      0.99      0.99      1843
weighted avg       0.99      0.99      0.99      1843

Validation Classification Report:
              precision    recall  f1-score   support

           0       0.94      0.93      0.93       185
           1       0.91      0.92      0.92       141

    accuracy                           0.93       326
   macro avg       0.92      0.93      0.93       326
weighted avg       0.93      0.93      0.93       326



In [ ]:
from torchviz import make_dot
embedding_dict = tokenizer.vocab_size
embedding_size = 128
hidden_size = 256
num_layers = 1
num_classes = 2
dropout = 0.4
learning_rate = 0.0007
num_epochs = 15

# Initialize the model, loss function and optimizer
model = CombinedLSTM(embedding_dict, embedding_size, hidden_size, num_layers, num_classes, dropout).to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

batch = next(iter(fw_train_loader))
scores = model(batch[0]['input_ids'])

print(model)
make_dot(scores, params=dict(list(model.named_parameters()))).render("model_images/sequence_combined_lstm", format="png")

c:\Users\a_sta\anaconda3\Lib\site-packages\torch\nn\modules\rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.4 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


CombinedLSTM(
  (embedding): Embedding(30522, 128)
  (dropout): Dropout(p=0.4, inplace=False)
  (lstm): LSTM(128, 256, batch_first=True, dropout=0.4)
  (attention): Linear(in_features=256, out_features=1, bias=True)
  (fc): Linear(in_features=256, out_features=2, bias=True)
)


'model_images\\sequence_combined_lstm.png'